In [1]:
import pandas as pd
import numpy as np
import re
from time import time
import math
import feather
from datetime import datetime
import os
import glob
# from tqdm import tqdm
import optuna
from optuna.integration import KerasPruningCallback
import tempfile
import tensorflow as tf
import random as rn

#Using keras
import keras
from keras import backend as K
from keras.models import Sequential  
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

#from scripts.utils import load_rnn_data
pd.options.display.max_columns = 200

# validation
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error, log_loss
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy

# others
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


## Load Data

In [2]:
def load_rnn_data(path, window, predict_ts, isdim3=True, geo_col=["geoid10_tract"], y_cols=["crime"]):
    """
    y_cols: ["crime"] or ["incident_type_0", "incident_type_1", "incident_type_2"]
    geo_col: ["geoid10_tract"] or ["geoid10_block"]
    return y_all and x_all of given path
    """
    # load data
    df = feather.read_dataframe(path)
    df.sort_values(by=["datetime", "geoid10_tract"], inplace=True)
    df.set_index("datetime", inplace=True)

    # input columns
    x_cols = list(df.drop(y_cols + geo_col, axis=1).columns)

    # group by geoid
    geo_grs = df.groupby(by=geo_col)

    # arrayes to store x and y
    # (no of timesteps, window size, no of tracts,  no of features, )
    n_timesteps = int(len(df) / len(geo_grs)) - window - predict_ts + 1
    x_all = np.empty(shape=(n_timesteps, window, len(geo_grs), len(x_cols + y_cols)))

    # (output size, no of tracts, no of outputs)
    y_all = np.empty(shape=(n_timesteps, len(geo_grs), len(y_cols)))

    # to store geo_ids and y_all's datetime
    geo_ids = []

    y_datetime = df.index.unique()[window + predict_ts - 1:]

    for i, (geo_id, gr) in enumerate(geo_grs):
        geo_ids.append(geo_id)
        x_values = gr[y_cols + x_cols].values
        y_values = gr[y_cols].values

        for j in range(window, len(gr) - predict_ts + 1):
            # generate x_all
            x_all[j - window, :, i, :] = x_values[j - window:j, :]
            y_all[j - window, i, :] = y_values[j + predict_ts - 1, :]

    if isdim3:
        x_all = np.reshape(x_all,
                           newshape=(x_all.shape[0], x_all.shape[1], x_all.shape[2] * x_all.shape[3]))
        y_all = np.reshape(y_all,
                           newshape=(y_all.shape[0], y_all.shape[1] * y_all.shape[2]))

    return x_all, y_all, geo_ids, y_datetime

In [3]:
# set configuration
path = "./features/features_binary_tract_2H.feather"
window = 12
predict_ts = 1  # how many timesteps future does the model predict? 

In [4]:
# load data as x and y of RNN
x_all, y_all, geo_ids, y_datetime = load_rnn_data(path=path,
                                                  window=window,
                                                  predict_ts=predict_ts,
                                                  isdim3=True,
                                                  geo_col=["geoid10_tract"],
                                                  y_cols=["crime"])

In [5]:
print(x_all.shape)
print(y_all.shape)
print(len(geo_ids))  # to convert model output later
print(len(y_datetime))  # to convert model output later

(18600, 12, 1560)
(18600, 195)
195
18600


## Modeling
see https://github.com/pfnet/optuna/blob/master/examples/pruning/keras_integration.py

In [6]:
BATCHSIZE = 128
EPOCHS = 200

In [7]:
def create_model(trial, x_train, y_train):
    # We optimize the number of layers, hidden units and dropout in each layer and
    # the learning rate of RMSProp optimizer.
    
    
    # We define our MLP.
#     n_layers = trial.suggest_int('n_layers', 1, 3)
    model = Sequential()
    model.add(LSTM(units=100, input_shape=(x_train.shape[1], x_train.shape[2])))
    #Hitting the first error above - how do we get the time series code to feed an example of 
#     for i in range(n_layers):
#         num_hidden = int(trial.suggest_loguniform('n_units_l{}'.format(i), 4, BATCHSIZE))
#         model.add(Dense(num_hidden, activation='sigmoid'))
    dropout = trial.suggest_uniform('dropout', 0.1, 0.5)
    model.add(Dropout(rate=dropout))
    model.add(Dense(units=y_train.shape[1], activation='sigmoid'))

    # We compile our model with a sampled learning rate.
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=lr),
                  metrics=['acc'])

    return model

In [8]:
## Need to define this to produce a training and test sample
def splitting(x_all, y_all):
    # create train and test set
    split = int(x_all.shape[0] * 0.8)   
    x_train = x_all[:split]
    y_train = y_all[:split]
    x_test = x_all[split:]
    y_test = y_all[split:]
    return x_train, y_train, x_test, y_test

In [9]:
#split = int(len(x_all) * 0.8)    
#x_train = x_all[:split]
#y_train = x_all[:split]
#x_test = x_all[split:]
#y_test = x_all[split:]

In [10]:
def objective(trial):
    # Clear clutter form previous session graphs.
    keras.backend.clear_session()
    
    # time series split
    x_train, y_train, x_test, y_test = splitting(x_all, y_all) 
    # Generate our trial model.
    model = create_model(trial, x_train, y_train)

    earlystopping = EarlyStopping(monitor="val_loss", patience=5)
    
    # Fit the model on the training data.
    # The KerasPruningCallback checks for pruning condition every epoch.
    model.fit(x_train,
              y_train,
              batch_size=BATCHSIZE,
              callbacks=[KerasPruningCallback(trial, 'val_acc'), earlystopping],
              epochs=EPOCHS,
              validation_split=0.2,
              shuffle=False,
              verbose=1)
    # Evaluate the model accuracy on the test set.
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[0]

In [11]:
# set seed
np.random.seed(0)
os.environ["PYTHONHASHSEED"] = "0"
rn.seed(0)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

study = optuna.create_study(direction="minimize",
                            pruner=optuna.pruners.MedianPruner()
                           )
study.optimize(objective, n_trials=100)
pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
print('Study statistics: ')
print('  Number of finished trials: ', len(study.trials))
print('  Number of pruned trials: ', len(pruned_trials))
print('  Number of complete trials: ', len(complete_trials))

print('Best trial:')
trial = study.best_trial



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 11904 samples, validate on 2976 samples
Epoch 1/200
11904/11904 [==============================] - ETA: 4:03 - loss: 0.7026 - acc: 0.535 - ETA: 2:42 - loss: 0.7039 - acc: 0.533 - ETA: 2:01 - loss: 0.7032 - acc: 0.534 - ETA: 1:38 - loss: 0.7037 - acc: 0.533 - ETA: 1:23 - loss: 0.7037 - acc: 0.533 - ETA: 1:15 - loss: 0.7035 - acc: 0.533 - ETA: 1:08 - loss: 0.7036 - acc: 0.533 - ETA: 1:03 - loss: 0.7036 - acc: 0.533 - ETA: 59s - loss: 0.7035 - acc: 0.534 - ETA: 55s - loss: 0.7035 - acc: 0.53 - ETA: 53s - loss: 0.7034 - acc: 0.53 - ETA: 50s - loss: 0.7035 - acc: 0.53 - ETA: 49s - loss: 0.7035 - acc: 0.53 - ETA: 47s - loss: 0.7032 - acc: 0.53 - ETA: 45s - loss: 0.7032 - acc: 0.53 - ETA: 44s - loss: 0.7029 - acc: 0.53 - ETA: 42s - loss: 0.7028 - acc: 0

Epoch 3/200
11904/11904 [==============================] - ETA: 31s - loss: 0.6587 - acc: 0.60 - ETA: 40s - loss: 0.6601 - acc: 0.60 - ETA: 36s - loss: 0.6592 - acc: 0.60 - ETA: 35s - loss: 0.6587 - acc: 0.60 - ETA: 34s - loss: 0.6585 - acc: 0.60 - ETA: 33s - loss: 0.6589 - acc: 0.60 - ETA: 32s - loss: 0.6592 - acc: 0.60 - ETA: 31s - loss: 0.6592 - acc: 0.60 - ETA: 31s - loss: 0.6591 - acc: 0.60 - ETA: 30s - loss: 0.6594 - acc: 0.60 - ETA: 29s - loss: 0.6590 - acc: 0.60 - ETA: 29s - loss: 0.6592 - acc: 0.60 - ETA: 28s - loss: 0.6592 - acc: 0.60 - ETA: 29s - loss: 0.6589 - acc: 0.60 - ETA: 28s - loss: 0.6587 - acc: 0.60 - ETA: 28s - loss: 0.6585 - acc: 0.60 - ETA: 27s - loss: 0.6584 - acc: 0.60 - ETA: 27s - loss: 0.6583 - acc: 0.60 - ETA: 27s - loss: 0.6582 - acc: 0.60 - ETA: 26s - loss: 0.6580 - acc: 0.60 - ETA: 26s - loss: 0.6578 - acc: 0.60 - ETA: 25s - loss: 0.6576 - acc: 0.60 - ETA: 25s - loss: 0.6568 - acc: 0.60 - ETA: 25s - loss: 0.6561 - acc: 0.60 - ETA: 24s - loss: 0.6559 - acc

11904/11904 [==============================] - ETA: 31s - loss: 0.6178 - acc: 0.66 - ETA: 31s - loss: 0.6192 - acc: 0.66 - ETA: 30s - loss: 0.6200 - acc: 0.66 - ETA: 29s - loss: 0.6190 - acc: 0.66 - ETA: 29s - loss: 0.6194 - acc: 0.66 - ETA: 30s - loss: 0.6197 - acc: 0.66 - ETA: 29s - loss: 0.6201 - acc: 0.66 - ETA: 29s - loss: 0.6201 - acc: 0.66 - ETA: 29s - loss: 0.6200 - acc: 0.66 - ETA: 28s - loss: 0.6202 - acc: 0.66 - ETA: 28s - loss: 0.6201 - acc: 0.66 - ETA: 27s - loss: 0.6205 - acc: 0.66 - ETA: 27s - loss: 0.6205 - acc: 0.66 - ETA: 26s - loss: 0.6205 - acc: 0.66 - ETA: 26s - loss: 0.6204 - acc: 0.66 - ETA: 27s - loss: 0.6201 - acc: 0.66 - ETA: 26s - loss: 0.6198 - acc: 0.66 - ETA: 26s - loss: 0.6197 - acc: 0.66 - ETA: 25s - loss: 0.6195 - acc: 0.66 - ETA: 25s - loss: 0.6193 - acc: 0.66 - ETA: 25s - loss: 0.6191 - acc: 0.66 - ETA: 24s - loss: 0.6189 - acc: 0.66 - ETA: 24s - loss: 0.6177 - acc: 0.66 - ETA: 23s - loss: 0.6167 - acc: 0.66 - ETA: 23s - loss: 0.6163 - acc: 0.66 - ETA

11904/11904 [==============================] - ETA: 29s - loss: 0.5782 - acc: 0.72 - ETA: 30s - loss: 0.5798 - acc: 0.72 - ETA: 31s - loss: 0.5820 - acc: 0.71 - ETA: 30s - loss: 0.5826 - acc: 0.71 - ETA: 29s - loss: 0.5838 - acc: 0.71 - ETA: 29s - loss: 0.5844 - acc: 0.71 - ETA: 28s - loss: 0.5854 - acc: 0.71 - ETA: 30s - loss: 0.5852 - acc: 0.71 - ETA: 29s - loss: 0.5850 - acc: 0.71 - ETA: 29s - loss: 0.5846 - acc: 0.71 - ETA: 29s - loss: 0.5843 - acc: 0.71 - ETA: 28s - loss: 0.5841 - acc: 0.71 - ETA: 28s - loss: 0.5843 - acc: 0.71 - ETA: 27s - loss: 0.5842 - acc: 0.71 - ETA: 27s - loss: 0.5843 - acc: 0.71 - ETA: 26s - loss: 0.5842 - acc: 0.71 - ETA: 26s - loss: 0.5839 - acc: 0.71 - ETA: 26s - loss: 0.5837 - acc: 0.71 - ETA: 25s - loss: 0.5835 - acc: 0.71 - ETA: 25s - loss: 0.5834 - acc: 0.71 - ETA: 25s - loss: 0.5833 - acc: 0.71 - ETA: 24s - loss: 0.5831 - acc: 0.71 - ETA: 24s - loss: 0.5818 - acc: 0.72 - ETA: 24s - loss: 0.5803 - acc: 0.72 - ETA: 23s - loss: 0.5800 - acc: 0.72 - ETA

11904/11904 [==============================] - ETA: 32s - loss: 0.5502 - acc: 0.76 - ETA: 31s - loss: 0.5543 - acc: 0.75 - ETA: 31s - loss: 0.5533 - acc: 0.76 - ETA: 30s - loss: 0.5517 - acc: 0.76 - ETA: 30s - loss: 0.5520 - acc: 0.76 - ETA: 29s - loss: 0.5525 - acc: 0.76 - ETA: 28s - loss: 0.5530 - acc: 0.76 - ETA: 28s - loss: 0.5531 - acc: 0.76 - ETA: 28s - loss: 0.5530 - acc: 0.76 - ETA: 27s - loss: 0.5532 - acc: 0.76 - ETA: 27s - loss: 0.5529 - acc: 0.76 - ETA: 26s - loss: 0.5530 - acc: 0.76 - ETA: 26s - loss: 0.5535 - acc: 0.76 - ETA: 26s - loss: 0.5538 - acc: 0.76 - ETA: 25s - loss: 0.5537 - acc: 0.76 - ETA: 25s - loss: 0.5534 - acc: 0.76 - ETA: 25s - loss: 0.5533 - acc: 0.76 - ETA: 25s - loss: 0.5534 - acc: 0.76 - ETA: 24s - loss: 0.5531 - acc: 0.76 - ETA: 24s - loss: 0.5528 - acc: 0.76 - ETA: 24s - loss: 0.5526 - acc: 0.76 - ETA: 23s - loss: 0.5523 - acc: 0.76 - ETA: 23s - loss: 0.5505 - acc: 0.76 - ETA: 23s - loss: 0.5489 - acc: 0.76 - ETA: 22s - loss: 0.5484 - acc: 0.76 - ETA

...

[I 2019-05-05 13:49:03,196] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 1.


Train on 11904 samples, validate on 2976 samples
Epoch 1/200
11904/11904 [==============================] - ETA: 2:07 - loss: 0.7007 - acc: 0.541 - ETA: 1:16 - loss: 0.6929 - acc: 0.558 - ETA: 59s - loss: 0.6850 - acc: 0.574 - ETA: 50s - loss: 0.6765 - acc: 0.59 - ETA: 44s - loss: 0.6678 - acc: 0.60 - ETA: 41s - loss: 0.6592 - acc: 0.62 - ETA: 38s - loss: 0.6505 - acc: 0.63 - ETA: 36s - loss: 0.6421 - acc: 0.65 - ETA: 35s - loss: 0.6338 - acc: 0.66 - ETA: 33s - loss: 0.6254 - acc: 0.68 - ETA: 32s - loss: 0.6174 - acc: 0.69 - ETA: 31s - loss: 0.6098 - acc: 0.70 - ETA: 30s - loss: 0.6023 - acc: 0.71 - ETA: 29s - loss: 0.5952 - acc: 0.72 - ETA: 29s - loss: 0.5882 - acc: 0.73 - ETA: 28s - loss: 0.5812 - acc: 0.74 - ETA: 27s - loss: 0.5746 - acc: 0.75 - ETA: 27s - loss: 0.5681 - acc: 0.75 - ETA: 26s - loss: 0.5616 - acc: 0.76 - ETA: 26s - loss: 0.5553 - acc: 0.77 - ETA: 25s - loss: 0.5492 - acc: 0.77 - ETA: 25s - loss: 0.5428 - acc: 0.78 - ETA: 24s - loss: 0.5339 - acc: 0.78 - ETA: 24s - lo

[I 2019-05-05 13:50:09,787] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 1.


Study statistics: 
('  Number of finished trials: ', 100)
('  Number of pruned trials: ', 90)
('  Number of complete trials: ', 10)
Best trial:


NameError: name 'trials' is not defined

In [12]:
print('  Value: ', trial.value)

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

('  Value: ', 0.29137174929341964)
  Params: 
    dropout: 0.159961594635
    lr: 1.77547438185e-05
